База с разметкой

In [314]:
import pandas as pd
data = pd.DataFrame()

for i in range(1, 7):
    data_part = pd.read_excel(f'women_cases_part_{i}.xlsx')
    data = pd.concat([data, data_part])

data = data[['ID', 'Статья УК РФ', 'text', 'ДН']].dropna(subset=['ДН', 'text']).drop_duplicates('ID').set_index('ID')
data.head()

,Статья УК РФ,text,ДН
ID,,,
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1.0
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1.0
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1.0
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1.0
6dd9a02fe15e9c2d12982ef8adbf8b4b,Статья 105 Часть 2 п.в,Дело № 2-5/18\nП Р И Г О В О Р\nименем Российс...,1.0


In [315]:
data['n_articles'] = data['Статья УК РФ'].apply(lambda x: len(x.split(',')))
data['art_105'] = data['Статья УК РФ'].str.contains('105').astype(int)
data['art_107'] = data['Статья УК РФ'].str.contains('107').astype(int)
data['art_111'] = data['Статья УК РФ'].str.contains('111 Часть 4').astype(int)
data.head()

,Статья УК РФ,text,ДН,n_articles,art_105,art_107,art_111
ID,,,,,,,
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1.0,1,1,0,0
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1.0,1,1,0,0
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1.0,1,1,0,0
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1.0,1,1,0,0
6dd9a02fe15e9c2d12982ef8adbf8b4b,Статья 105 Часть 2 п.в,Дело № 2-5/18\nП Р И Г О В О Р\nименем Российс...,1.0,1,1,0,0


In [316]:
len(data)

756

In [317]:
data['ДН'].value_counts(normalize=True)

1.0     0.448413
99.0    0.269841
0.0     0.264550
98.0    0.017196
Name: ДН, dtype: float64

In [344]:
data[data['ДН']<98]['ДН'].value_counts(normalize=True)

1.0    0.628942
0.0    0.371058
Name: ДН, dtype: float64

Проверяем дела, ошибочно определённые как "женские"

In [236]:
import re
patterns = [
    r'скончалась',
    r'погибш[аеу][йяю]',
    r'смерт[иь]ю? потерпевшей',
    r'смерт[иь]ю? последней',
    r'мертва',
    r'умерла',
    r'е[её] труп',
    r'труп женщины',
    r'оказывала сопротивлени',
    r'изби[лть]+ е[её]',
    r'е[её] изби[лть]+',
    r'уби[лть]+ е[её]',
    r'е[её] уби[лть]+',
    r'нан[её]с ей [<\d>]+? удар'
]

def check_patterns(text):
    global patterns
    
    for pattern in patterns:
        if re.search(pattern, text):
            return True
    return False

def define_pattern(text):
    global patterns
    
    for pattern in patterns:
        if re.search(pattern, text):
            return pattern
    return None

In [237]:
# приняли решение убрать из ключевых слов первые 6 выражений, кроме "её труп"
data[data['ДН']==99]['text'].apply(define_pattern).value_counts()

уби[лть]+ е[её]              51
умерла                       45
изби[лть]+ е[её]             30
е[её] изби[лть]+             24
е[её] труп                   18
е[её] уби[лть]+              13
скончалась                   11
смерт[иь]ю? потерпевшей       3
погибш[аеу][йяю]              2
мертва                        2
нан[её]с ей [<\d>]+? удар     2
оказывала сопротивлени        1
труп женщины                  1
смерт[иь]ю? последней         1
Name: text, dtype: int64

In [111]:
for text in data[(data['ДН']==99)&(data['text'].str.lower().str.contains('умерла'))]['text'][:7]:
    print(text)

П Р И Г О В О Р
ИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ
Дело № 1-17/2018                             15 марта 2018 года 
Пушкинский районный суд Санкт-Петербурга в составе 
председательствующего судьи Макаровой Т.Г.,
при секретарях Стафеевой Ю.В., Белоусе А.О.,
с участием помощников прокурора Пушкинского района Санкт-Петербурга Коломийца В.В., Пройдаковой А.В., Серобяна А.В., Смирновой О.О., прокурора Пушкинского района Санкт-Петербурга Никитиной И.Н.,
потерпевшего П,
подсудимого Никифорова Д.В., 
защитника адвоката Абакумовой Т.И., представившей удостоверение № 1922 и ордер Н 099408 от 12.10.2017 года,
рассмотрев материалы уголовного дела в отношении: 
Никифоров Д.В. , ..., ранее судимого:
- 24.04.2009 года Лужским городским судом Ленинградской области по ч. 3 ст. 30 ч. 2 ст. 167, ч. 1 ст. 105, ч. 4 ст. 111 УК РФ на основании ч. 3 ст. 69 УК РФ к 12 годам лишения свободы, с отбыванием наказания в исправительной колонии строгого режима, постановлением Тосненского городского суда Ленинградской облас

Предобрабатываем текстовые данные для модели

In [318]:
data = data[data['ДН'] < 98]
data['text_prep'] = data['text'].str.lower()
data.head()

,Статья УК РФ,text,ДН,n_articles,art_105,art_107,art_111,text_prep
ID,,,,,,,,
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1.0,1,1,0,0,дело № 2-8/2018 приговор\nименем российской фе...
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1.0,1,1,0,0,2-26/2018\nприговор\nименем российской федерац...
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1.0,1,1,0,0,дело № 2-8/2018\nприговор\nименем российской ф...
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1.0,1,1,0,0,дело <номер обезличен>\nприговор\nименем росси...
6dd9a02fe15e9c2d12982ef8adbf8b4b,Статья 105 Часть 2 п.в,Дело № 2-5/18\nП Р И Г О В О Р\nименем Российс...,1.0,1,1,0,0,дело № 2-5/18\nп р и г о в о р\nименем российс...


In [319]:
def keep_only_rus(text):    
    new_text = ''
    for symbol in text:
        if re.match(r'[А-я]', symbol) or symbol == ' ':
            new_text += symbol
        else:
            new_text += ' '
    return new_text

In [320]:
data['text_prep'] = data['text_prep'].apply(keep_only_rus)
data.head()

,Статья УК РФ,text,ДН,n_articles,art_105,art_107,art_111,text_prep
ID,,,,,,,,
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1.0,1,1,0,0,дело приговор именем российской фед...
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1.0,1,1,0,0,приговор именем российской федерации...
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1.0,1,1,0,0,дело приговор именем российской фед...
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1.0,1,1,0,0,дело номер обезличен приговор именем российс...
6dd9a02fe15e9c2d12982ef8adbf8b4b,Статья 105 Часть 2 п.в,Дело № 2-5/18\nП Р И Г О В О Р\nименем Российс...,1.0,1,1,0,0,дело п р и г о в о р именем российско...


In [321]:
def del_double_spaces(text_with_double_spaces):
    while '  ' in text_with_double_spaces:
        text_with_double_spaces = text_with_double_spaces.replace('  ',' ')
    return text_with_double_spaces

In [322]:
data['text_prep'] = data['text_prep'].apply(del_double_spaces)
data.head()

,Статья УК РФ,text,ДН,n_articles,art_105,art_107,art_111,text_prep
ID,,,,,,,,
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1.0,1,1,0,0,дело приговор именем российской федерации г ек...
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1.0,1,1,0,0,приговор именем российской федерации г екатер...
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1.0,1,1,0,0,дело приговор именем российской федерации г сы...
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1.0,1,1,0,0,дело номер обезличен приговор именем российско...
6dd9a02fe15e9c2d12982ef8adbf8b4b,Статья 105 Часть 2 п.в,Дело № 2-5/18\nП Р И Г О В О Р\nименем Российс...,1.0,1,1,0,0,дело п р и г о в о р именем российской федерац...


In [323]:
import pymystem3
mstem = pymystem3.Mystem()

def lemmatize(raw_text):
    return ''.join(mstem.lemmatize(raw_text)).strip()

In [324]:
data['text_prep'] = data['text_prep'].apply(lemmatize)
data.head()

,Статья УК РФ,text,ДН,n_articles,art_105,art_107,art_111,text_prep
ID,,,,,,,,
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1.0,1,1,0,0,дело приговор имя российский федерация г екате...
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1.0,1,1,0,0,приговор имя российский федерация г екатеринбу...
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1.0,1,1,0,0,дело приговор имя российский федерация г сыкты...
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1.0,1,1,0,0,дело номер обезличивать приговор имя российски...
6dd9a02fe15e9c2d12982ef8adbf8b4b,Статья 105 Часть 2 п.в,Дело № 2-5/18\nП Р И Г О В О Р\nименем Российс...,1.0,1,1,0,0,дело п р и г о в о р имя российский федерация ...


In [325]:
import stop_words

stopwords = stop_words.get_stop_words('russian')
stopwords.extend(stop_words.get_stop_words('english'))
stopwords = list(set(stopwords))
stopwords += [
    'фио', 'гггг', 'подсудимый', 'суд',
    'изымать', 'согласно', 'наказание',
    'потерпевший', 'показание', 'судебный',
    'преступление', 'адрес', 'свидетель',
    'свой', 'находиться', 'час', 'ход'
             ]

def del_stopwords(text):
    global stopwords
    new_text = []
    for word in text.split():
        if word not in stopwords and len(word) > 2:
            new_text.append(word)
    return ' '.join(new_text)

In [326]:
data['text_prep'] = data['text_prep'].apply(del_stopwords)
data.head()

,Статья УК РФ,text,ДН,n_articles,art_105,art_107,art_111,text_prep
ID,,,,,,,,
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1.0,1,1,0,0,дело приговор российский федерация екатеринбур...
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1.0,1,1,0,0,приговор российский федерация екатеринбург авг...
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1.0,1,1,0,0,дело приговор российский федерация сыктывкар с...
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1.0,1,1,0,0,дело номер обезличивать приговор российский фе...
6dd9a02fe15e9c2d12982ef8adbf8b4b,Статья 105 Часть 2 п.в,Дело № 2-5/18\nП Р И Г О В О Р\nименем Российс...,1.0,1,1,0,0,дело российский федерация апрель областной сос...


In [327]:
data['text_prep'] = data['text_prep'].apply(del_double_spaces)
data.head()

,Статья УК РФ,text,ДН,n_articles,art_105,art_107,art_111,text_prep
ID,,,,,,,,
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1.0,1,1,0,0,дело приговор российский федерация екатеринбур...
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1.0,1,1,0,0,приговор российский федерация екатеринбург авг...
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1.0,1,1,0,0,дело приговор российский федерация сыктывкар с...
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1.0,1,1,0,0,дело номер обезличивать приговор российский фе...
6dd9a02fe15e9c2d12982ef8adbf8b4b,Статья 105 Часть 2 п.в,Дело № 2-5/18\nП Р И Г О В О Р\nименем Российс...,1.0,1,1,0,0,дело российский федерация апрель областной сос...


In [328]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(min_df=0.05).fit(data['text_prep'])
matrix = cvect.transform(data['text_prep'])
td_matrix = pd.DataFrame(matrix.toarray(), index=data.index, columns=cvect.get_feature_names())
td_matrix.head()

,абонент,абонентский,ава,август,автомашина,автомобиль,агрессивный,агрессия,адвокат,адвокатский,...,явление,являться,явно,явный,ягодица,язык,якобы,январь,яремный,ящик
ID,,,,,,,,,,,,,,,,,,,,,
3b8b13f92c16de8465ace460479d93ef,0,0,0,1,0,0,0,0,2,0,...,0,6,0,0,0,0,0,5,2,0
2a764fb2100c742f21b0deadd564652f,0,0,0,1,0,0,2,0,3,0,...,0,6,1,0,0,0,0,0,0,1
a6feaaa7ad297d38f5ff8056056fad5b,0,0,0,0,0,0,2,0,1,0,...,0,7,0,0,0,0,0,0,0,0
8ea1e5df6b2f5c5ebba9862ce39426bb,1,1,0,0,0,0,4,1,2,0,...,2,3,0,0,0,7,0,0,0,0
6dd9a02fe15e9c2d12982ef8adbf8b4b,0,2,0,0,0,2,0,1,2,1,...,0,6,0,0,0,0,1,0,0,0


In [329]:
td_matrix.sum().sort_values(ascending=False)[:30]

нож               9383
повреждение       8681
смерть            6778
область           6739
потерпеть         6631
удар              6422
место             6345
дом               6323
данные            6140
кровь             5787
квартира          5706
рука              5418
левый             5348
обнаруживать      4939
труп              4918
рана              4881
действие          4851
обстоятельство    4839
правый            4784
совершение        4695
становиться       4667
состояние         4666
заседание         4537
наносить          4505
дело              4486
причинять         4337
поверхность       4257
убийство          4153
минута            4066
цвет              4050
dtype: int64

In [331]:
td_matrix['VIOLENCE'] = data['ДН']
td_matrix['N_ARTICLES'] = data['n_articles']
td_matrix['A_105'] = data['art_105']
td_matrix['A_107'] = data['art_107']
td_matrix['A_111'] = data['art_111']
td_matrix

,абонент,абонентский,ава,август,автомашина,автомобиль,агрессивный,агрессия,адвокат,адвокатский,...,язык,якобы,январь,яремный,ящик,VIOLENCE,N_ARTICLES,A_105,A_107,A_111
ID,,,,,,,,,,,,,,,,,,,,,
3b8b13f92c16de8465ace460479d93ef,0,0,0,1,0,0,0,0,2,0,...,0,0,5,2,0,1.0,1,1,0,0
2a764fb2100c742f21b0deadd564652f,0,0,0,1,0,0,2,0,3,0,...,0,0,0,0,1,1.0,1,1,0,0
a6feaaa7ad297d38f5ff8056056fad5b,0,0,0,0,0,0,2,0,1,0,...,0,0,0,0,0,1.0,1,1,0,0
8ea1e5df6b2f5c5ebba9862ce39426bb,1,1,0,0,0,0,4,1,2,0,...,7,0,0,0,0,1.0,1,1,0,0
6dd9a02fe15e9c2d12982ef8adbf8b4b,0,2,0,0,0,2,0,1,2,1,...,0,1,0,0,0,1.0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
036cd15cd97db731f98892900276711e,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,1.0,2,1,0,0
c2aec36e9323ea7874bb3bd193e06743,0,0,0,2,0,0,0,1,1,0,...,0,0,1,0,0,1.0,1,1,0,0
177f876e32f9c3fea46efefd1715fc74,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0.0,2,1,0,0


In [332]:
y = td_matrix['VIOLENCE']
X = td_matrix.drop("VIOLENCE", axis=1)
y.shape, X.shape

((539,), (539, 2614))

In [333]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=4)

In [334]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=15, min_samples_leaf=5, min_samples_split=5, random_state=4)
rfc.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=5, min_samples_split=5, n_estimators=15,
                       random_state=4)

In [335]:
from sklearn.metrics import classification_report
print(classification_report(y_train, rfc.predict(X_train)))
print(classification_report(y_test, rfc.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.99      0.80      0.88       181
         1.0       0.89      0.99      0.94       304

    accuracy                           0.92       485
   macro avg       0.94      0.90      0.91       485
weighted avg       0.93      0.92      0.92       485

              precision    recall  f1-score   support

         0.0       0.43      0.16      0.23        19
         1.0       0.66      0.89      0.76        35

    accuracy                           0.63        54
   macro avg       0.54      0.52      0.49        54
weighted avg       0.58      0.63      0.57        54



In [336]:
import numpy as np
import operator
feat_indexes = np.argsort(rfc.feature_importances_)[::-1]
feat_words = {v:k for k, v in cvect.vocabulary_.items()}
[print(feat_words[i]) for i in feat_indexes[:17]];

подсудимая
сумма
перчатка
деньги
основание
дом
забирать
арест
денежный
цель
распитие
средство
давать
гражданка
находить
жилище
ревновать


In [380]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(max_depth=3, n_estimators=50, random_state=4)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=50, random_state=4)

In [381]:
print(classification_report(y_train, gbc.predict(X_train)))
print(classification_report(y_test, gbc.predict(X_test)))

              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96       181
         1.0       0.96      1.00      0.98       304

    accuracy                           0.97       485
   macro avg       0.98      0.96      0.97       485
weighted avg       0.97      0.97      0.97       485

              precision    recall  f1-score   support

         0.0       0.71      0.63      0.67        19
         1.0       0.81      0.86      0.83        35

    accuracy                           0.78        54
   macro avg       0.76      0.74      0.75        54
weighted avg       0.77      0.78      0.77        54



In [373]:
feat_indexes = np.argsort(gbc.feature_importances_)[::-1]
feat_words = {v:k for k, v in cvect.vocabulary_.items()}
[print(feat_words[i]) for i in feat_indexes[:17]];

подсудимая
ревность
жилище
деньги
мать
хищение
опьянение
крыльцо
очный
медико
обвиняемый
погибший
квартира
член
ревновать
размер
ранее


In [340]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0, max_iter=10000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=10000, random_state=0)

In [341]:
print(classification_report(y_train, lr.predict(X_train)))
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       181
         1.0       1.00      1.00      1.00       304

    accuracy                           1.00       485
   macro avg       1.00      1.00      1.00       485
weighted avg       1.00      1.00      1.00       485

              precision    recall  f1-score   support

         0.0       0.64      0.47      0.55        19
         1.0       0.75      0.86      0.80        35

    accuracy                           0.72        54
   macro avg       0.70      0.67      0.67        54
weighted avg       0.71      0.72      0.71        54



In [342]:
pd.DataFrame(zip(list(X.columns), list(lr.coef_[0])), columns=['word', 'coef']).sort_values('coef', ascending=False)[:30]

,word,coef
2151,сожительница,0.262971
469,дочь,0.238803
881,мать,0.202564
425,доказательство,0.184399
688,исследовать,0.179295
134,видеть,0.173385
2110,смыв,0.170764
1922,ревность,0.167830
1751,причинение,0.164977
1075,никто,0.160789


In [343]:
pd.DataFrame(zip(list(X.columns), list(lr.coef_[0])), columns=['word', 'coef']).sort_values('coef', ascending=True)[:30]

,word,coef
1482,подсудимая,-0.282937
2589,экспертиза,-0.234761
381,деньги,-0.200647
601,здоровье,-0.183559
1183,одежда,-0.173963
2203,спиртное,-0.141382
379,дело,-0.140538
896,местность,-0.140226
4,автомашина,-0.138691
1011,находить,-0.133583


Полная база текстов, предсказание неразмеченных кейсов

In [345]:
full = pd.read_excel('women_cases_ALL.xlsx')[['ID', 'Статья УК РФ', 'text']].set_index('ID')
full['n_articles'] = full['Статья УК РФ'].apply(lambda x: len(x.split(',')))
full['art_105'] = full['Статья УК РФ'].str.contains('105').astype(int)
full['art_107'] = full['Статья УК РФ'].str.contains('107').astype(int)
full['art_111'] = full['Статья УК РФ'].str.contains('111 Часть 4').astype(int)
full.head()

,Статья УК РФ,text,n_articles,art_105,art_107,art_111
ID,,,,,,
ec89b941970471362dc925f201c91098,Статья 105 Часть 2 п.г; Статья 105 Часть 2 п.г,Дело № 2-31/2018\nП Р И Г О В О Р\nИменем Росс...,1,1,0,0
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1,1,0,0
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1,1,0,0
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1,1,0,0
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1,1,0,0


In [346]:
full['text_prep'] = full['text'].str.lower()
full['text_prep'] = full['text_prep'].apply(keep_only_rus)
full['text_prep'] = full['text_prep'].apply(del_double_spaces)
full['text_prep'] = full['text_prep'].apply(lemmatize)
full['text_prep'] = full['text_prep'].apply(del_stopwords)
full['text_prep'] = full['text_prep'].apply(del_double_spaces)
full.head()

,Статья УК РФ,text,n_articles,art_105,art_107,art_111,text_prep
ID,,,,,,,
ec89b941970471362dc925f201c91098,Статья 105 Часть 2 п.г; Статья 105 Часть 2 п.г,Дело № 2-31/2018\nП Р И Г О В О Р\nИменем Росс...,1,1,0,0,дело российский федерация октябрь город казань...
3b8b13f92c16de8465ace460479d93ef,Статья 105 Часть 2 п.в,Дело № 2-8/2018 ПРИГОВОР\nИменем Российской Фе...,1,1,0,0,дело приговор российский федерация екатеринбур...
2a764fb2100c742f21b0deadd564652f,Статья 105 Часть 2 п.ж,2-26/2018\nПРИГОВОР\nименем Российской Федерац...,1,1,0,0,приговор российский федерация екатеринбург авг...
a6feaaa7ad297d38f5ff8056056fad5b,Статья 105 Часть 2 п.д,Дело № 2-8/2018\nПРИГОВОР\nименем Российской Ф...,1,1,0,0,дело приговор российский федерация сыктывкар с...
8ea1e5df6b2f5c5ebba9862ce39426bb,Статья 105 Часть 2 п.в,Дело <Номер обезличен>\nПРИГОВОР\nименем Росси...,1,1,0,0,дело номер обезличивать приговор российский фе...


In [347]:
full_matrix = cvect.transform(full['text_prep'])
full_td_matrix = pd.DataFrame(full_matrix.toarray(), index=full.index, columns=cvect.get_feature_names())
full_td_matrix.shape

(2053, 2610)

In [348]:
full_td_matrix['N_ARTICLES'] = full['n_articles']
full_td_matrix['A_105'] = full['art_105']
full_td_matrix['A_107'] = full['art_107']
full_td_matrix['A_111'] = full['art_111']
full_td_matrix.shape

(2053, 2614)

In [383]:
predictions = pd.DataFrame(gbc.predict(full_td_matrix), index=full_td_matrix.index, columns=['prediction'])
predictions['probability_0'] = gbc.predict_proba(full_td_matrix)[:, 0]
predictions['probability_1'] = gbc.predict_proba(full_td_matrix)[:, 1]
predictions

,prediction,probability_0,probability_1
ID,,,
ec89b941970471362dc925f201c91098,1.0,0.283357,0.716643
3b8b13f92c16de8465ace460479d93ef,1.0,0.197964,0.802036
2a764fb2100c742f21b0deadd564652f,1.0,0.204232,0.795768
a6feaaa7ad297d38f5ff8056056fad5b,1.0,0.145239,0.854761
8ea1e5df6b2f5c5ebba9862ce39426bb,1.0,0.128135,0.871865
...,...,...,...
6f42bca4eb76fcd6acdfcf216edb1294,1.0,0.335740,0.664260
ce8b099236e8ae205ad95bc76e9bda54,1.0,0.231158,0.768842
3858ab19933c1dd4367f6b1ef0563cb2,0.0,0.529810,0.470190


In [386]:
predictions['prediction'].sum()*0.81 / len(predictions)

0.6079931807111544

In [387]:
predictions['prediction'].sum()*0.81 

1248.21

In [385]:
predictions.sum()

prediction       1541.0000
probability_0     737.0206
probability_1    1315.9794
dtype: float64